## Prerequisites

### SyftUI
Make sure you have SyftUI running with your email logged in.
1. It could be downloaded from [SyftUI](https://github.com/OpenMined/SyftUI/releases)


### Install dependencies for this demo

### Python Environment with Jupyter

Please make sure to have a virtual python environment with `jupyter` installed to run this notebook, e.g. run the below commands in a terminal
```bash
uv venv
source .venv/bin/activate
uv pip install jupyterlab
```

Configure the notebook to run with the Python kernel from the installed environment, or run `uv run jupyterlab` 

In [ ]:
!uv pip install syftbox-enclave syft-rds==0.1.1-dev.1

In [ ]:
from syft_rds import init_session
from syftbox_enclave.client import connect

## Explore Datasets

In [ ]:
DO1 = "do1@openmined.org"
DO2 = "do2@openmined.org"

do_client_1 = init_session(host=DO1)
print("Logged into: ", do_client_1.host)

do_client_2 = init_session(host=DO2)
print("Logged into: ", do_client_2.host)


In [ ]:
DATASET_NAME = "<DATASET_NAME>"

dataset1 = do_client_1.dataset.get(name=DATASET_NAME)
dataset1.describe()

In [ ]:
dataset2 = do_client_2.dataset.get(name=DATASET_NAME)
dataset2.describe()

### DO Job

In [ ]:
# Job submission
datasites = [do_client_1, do_client_2]

for client in datasites:
    job = client.jobs.submit(
            name="Crop Avg Experiment",
            description="Farming Coop Avg Experiment",
            user_code_path="./code",
            dataset_name=DATASET_NAME,
            tags=["enclave", "syft"],
            entrypoint="entrypoint.py",
            enclave = "enclave-organic-coop@openmined.org"
        )
    print(job)

### Enclave Client

In [ ]:
enclave_client = connect("enclave-organic-coop@openmined.org")

In [ ]:
proj_res = enclave_client.create_project(
                   project_name = "CropAvg",
                   datasets = [dataset1,dataset2] ,
                   output_owners = [DO1,DO2],
                   code_path = "./code",
                   entrypoint = "entrypoint.py"
            )
            

In [ ]:
proj_res.output()